<h1>Chapter 8: Dimensionality Reduction</h1>
<h2>8.1 Introduction</h2>
Many ML problems involve thousands or even million of features for each training instance. Not only do these features make training extremely slow, but they can also make it harder to find a good solution. This problem is often referred to as <em>the curse of dimensionality</em>.

Fortunately, in real world problems, it's often possible to reduce the number of features considerably. Apart from speding up training, dimensionality reduction is also extremely usefu for data visualisation. Reducing the number of dimensions down to two (or three) makes it possible to plot a condensed view of a high-dimensional training set on a graph .